In [ ]:
import re
from ltlcross_runner import param_runner, LtlcrossRunner
from tools_hier import get_tools, tool_order
from evaluation_utils import split_cols, sort_by_tools, to_tuples

## Helper functions to format the latex tables

In [ ]:
def add_type_lines(filename,Safra=True,vertical=False,end=None):
    cline = '\cmidrule[\lightrulewidth]{2-'+ str(end) +'}' \
        if vertical else '\midrule'
    todo = [(r'(\\multirow\{2\}\{\*\}\{ltl2dpa\})',''),
            (r'(\\multirow\{3\}\{\*\}\{ltl2dstar\})','')]
    if not Safra:
        todo = [todo[0]]
    if vertical:
        todo = [(r"&\s+"+t[0], ' & ') for t in todo]
    for rplc, pref in todo:
        with open(filename) as f:
            lines = f.read()
            lines = re.sub(rplc,cline + pref + r'\1',lines)
        with open(filename,"w") as f1:
            f1.write(lines)

In [ ]:
def fix_lines(filename, end=12, vertical=False):
    end = str(end)
    todo = [(r"(\\multirow\{2\}\{\*\}\{ltl2dpa\})",1,''),
        (r"(& \\multirow\{2\}\{\*\}\{Spot\})",2,'')]
    if vertical:
        todo = [(r"&\s+"+t[0],t[1]+1, ' & ') for t in todo]
    for rplc, start, pref in todo:
        with open(filename) as f:
            lines = f.read()
        lines = re.sub(rplc,
                       "\cmidrule{" +
                       "{}-{}".format(start,end) +
                       "}" + pref + r"\1",
                       lines)
        with open(filename,"w") as f1:
            f1.write(lines)
    with open(filename) as f:
        lines = f.read()
    lines = lines.replace('cline','cmidrule')
    lines = lines.replace('$nan$','---')
    if vertical:
        rplc = "\cmidrule{1-" + str(end) + "}"
        lines = lines.replace(rplc,"\midrule[\heavyrulewidth]")
    # Remove cmidrules in front of midrule
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\n?)*\\midrule\[\\heavyrulewidth\](\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", "\\midrule[\\heavyrulewidth] ", lines)
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\s*\n?)*(\\cmidrule\[\\lightrulewidth\]\{\d+-\d+\})(\s*\n*\s*\\cmidrule\{\d+-\d+\}\s*\n?)*", r"\2 ", lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def fix_tools(tool):
    tool = tool.replace('ltl2dstar(NBA)',
                        '\parbox[c]{1.3cm}{\centering ltl2dstar NBA}')
    if tool == 'ltl2dstar':
        tool = '\parbox[c]{1.3cm}{\centering ltl2dstar LTL}'
    tool = tool.replace('Spot',
                        #'\rotatebox[origin=c]{90}{Spot (autfilt)}')
                        '\parbox[c]{1.3cm}{\centering Spot autfilt}')
    tool = tool.replace('R3','Rabinizer 3').replace('R4','Rabinizer 4')
    tool = tool.replace('ltl2tgba','Spot')
    return tool

In [ ]:
def color_table(filename, extrarowheight='.75ex'):
    setup = '''\\newcolumntype{{a}}{{>{{\\columncolor{{blue!20}}}}r}}
\\newcolumntype{{b}}{{>{{\\columncolor{{blue!20}}}}c}}
\\setlength{{\\aboverulesep}}{{0pt}}
\\setlength{{\\belowrulesep}}{{0pt}}
\\setlength{{\\extrarowheight}}{{{}}}
\\setlength{{\\heavyrulewidth}}{{2pt}}
\\setlength{{\\lightrulewidth}}{{1.2pt}}
\\def\\high{{\\cellcolor{{darkgreen!40}}}}
'''.format(extrarowheight)
    with open(filename) as f:
        lines = f.read()
    lines = fix_header_colors(lines)
    with open(filename,"w") as f1:
        f1.write(setup + lines)

In [ ]:
def make_heading(filename):
    with open(filename) as f:
        lines = f.read()
    lines = re.sub(r"(\s+)&\s+&\s+&\s+states", 
        r"\1main tool & intermediate & acc & states",
        lines)
    lines = re.sub(r"(\s+)&\s+&\s+&\s+&\s+0", 
        r"\1main tool & intermediate & acc & \# & 0",
        lines)
    lines = re.sub(r"(\s+)&\s+&\s+&\s+\$n",
        r"\1main tool & intermediate & acc & $n",
        lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

## Get the edge where all tools succeed

In [ ]:
def param_get_edge_n(self,only_timeout=True,tool_subset=None):
    if tool_subset is None:
        tool_subset = self.tools.keys()
    v = self.values.states[list(tool_subset)]
    max_n = len(v.dropna())
    if only_timeout:
        es = self.exit_status.iloc[max_n]
        while len(es[es == 'timeout']) == 0:
            max_n += 1
            es = r.exit_status.iloc[max_n]
    return max_n

## Compute n for each tool

In [ ]:
def param_get_max_n(self,tool):
    v = self.values.states[tool]
    max_n = len(v.dropna())
    return max_n

## get_nX_states

In [ ]:
def param_get_value_for(self,tool,n,col='states'):
    v = self.values[col][tool]
    return v.iloc[n-1]

## Prepare table

In [ ]:
import pandas as pd

In [ ]:
def my_highlight_best_min(data):
    """highlight the minimum of a row."""
    return ['background-color: #3db709' if m == data.min() else ''
            for m in data]

def my_highlight_best_max(data):
    """highlight the maxinum of a row."""
    return ['background-color: #3db709' if m == data.max() else ''
            for m in data]

def my_highlight_max(data):
    """highlight the maximum of a row, and also highlight values that are not far away."""
    m = data.max()
    is_max = data == m
    is_good = (data >= (m/1.05)) | (data >= m-0.005)
    return ['background-color: #ff5050' if m else
            'background-color: #ff9999' if g else ''
            for m,g in zip(is_max, is_good)]

In [ ]:
#maxn = "max $n$"
maxn = "max"
n_str = '$n = {}$'
ne_str = '$n_e = {}$'
nt_str = '$n_t = {}$'
def compute_param(self,col='states',tool_subset=None):
    if tool_subset is None:
        tool_subset = self.tools.keys()
    nt = param_get_edge_n(self,tool_subset=tool_subset)
    ne = param_get_edge_n(self,False,tool_subset=tool_subset)
    
    if ne == nt:
        n_i = n_str.format(ne)
        cols = [n_i,maxn]
    else:
        ne_i = ne_str.format(ne)
        nt_i = nt_str.format(nt)
        cols = [ne_i,nt_i,maxn]
    idx = pd.Index(data=cols)
    df = pd.DataFrame(columns=tool_subset,index=idx)

    v = self.values[col]
    if ne == nt:
        df.loc[n_i] = v[df.columns].iloc[ne-1]
    else:
        df.loc[nt_i] = v[df.columns].iloc[nt-1]
        df.loc[ne_i] = v[df.columns].iloc[ne-1]
    for tool in tool_subset:
        df.loc[maxn].loc[tool] = float(param_get_max_n(self,tool))
    #tmp = df.copy()
    return df#tmp.style.apply(my_highlight_min, axis=1)

In [ ]:
ltlgux = ['gh-e','gh-c1','gh-c2','gh-q','gh-r',
          'gh-s','gh-u','gh-u2','ms-phi-r',
          'ms-phi-s','and-fg','or-fg','go-theta']
full = ['kr-n','kr-nlogn','ms-phi-h']

In [ ]:
','.join(ltlgux)

In [ ]:
data = []
for f in ltlgux:
    r = param_runner(f,get_tools('ltl-gux'))
    r.parse_results()
    data.append(compute_param(r))
    
for f in full:
    r = param_runner(f,get_tools('full'))
    r.parse_results()
    data.append(compute_param(r))

res = pd.concat(data,keys=ltlgux+full)

In [ ]:
def highlight_best(res, latex=False):  
    if latex:
        res = res.T
    # Choose all rows for numbers of states
    a = res.index.levels[1]
    for_min = a.delete(a.get_loc(maxn))

    idx = pd.IndexSlice
    # Color the best values
    if not latex:
        return res.style\
            .apply(my_highlight_best_min, axis=1,subset=idx[idx[:,for_min],:])\
            .apply(my_highlight_best_max, axis=1,subset=idx[idx[:,maxn],:])
    else:
        res.loc[idx[idx[:,for_min],:]] = res.loc[idx[idx[:,for_min],:]].astype(float)\
            .apply(high_min, axis=1)
        res.loc[idx[idx[:,maxn],:]] = res.loc[idx[idx[:,maxn],:]].astype(float)\
            .apply(high_max, axis=1)
        return res.T

In [ ]:
def high_min(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.min() else m for m in data]

def high_max(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.max() else m for m in data]

In [ ]:
h = highlight_best(res)
h

In [ ]:
res = sort_by_tools(res,tool_order,axis=1)
split = split_cols(res)
i = [(fix_tools(t[0]),t[1],t[2].replace('TEL.TP','TEL.TEL'))\
         for t in split.columns.values]
split.columns=pd.MultiIndex.from_tuples(i)

In [ ]:
def color_str_multiindex_cols(df, color_type='a', def_type='r'):
    col_f = '' # string to acummulate the columns' types
    i = 0 # iterates over labels
    c = 0 # remembers if this one should be colored
    while i < len(df.columns):
        col = df.columns.labels[0][i]
        current_lab = df.columns.levels[0][col]
        col_nums = len(df[current_lab].columns)
        i += col_nums # moves to next column
        if c % 2 == 0:
            col_f += color_type*col_nums
        else:
            col_f += def_type*col_nums
        if len(df[current_lab]) > 0:
            c += 1
    return col_f

In [ ]:
def param_to_latex(res, filename='param.tex', transpose=False, color=True):
    if transpose:
        res = res.T
        col_f = color_str_multiindex_cols(res) if color else 'r'*len(res.columns)
        col_f = 'ccr' + col_f
    else:
        color = False
        col_f = 'cc' + 'r'*len(res.columns)
    if color:
        res = highlight_best(res, latex=True)
    res.to_latex(buf=open(filename,'w'), multirow=True, escape=False,
                 na_rep='---', float_format=lambda x: '$' + '%0.0f' % x + '$',
                 column_format=col_f, multicolumn_format='c')
    if color:
        color_table(filename,'.2ex')
    if transpose:
        length = len(res.columns) + len(res.index.levels)
        fix_lines(filename,length)
        add_type_lines(filename)
        make_heading(filename)

In [ ]:
groups = {
'gh' : ['gh-e','gh-c1','gh-c2','gh-q'],
'gh2' : ['gh-r','gh-s','gh-u','gh-u2'],
'ms+go' : ['go-theta','ms-phi-h','ms-phi-r','ms-phi-s'],
'dexp+other' : ['kr-n','kr-nlogn','and-fg','or-fg'],
}

In [ ]:
for name, group in groups.items():
    param_to_latex(split.loc(axis=0)[group],
                   filename='param_{}.tex'.format(name),
                   transpose=True)
#    print('''\\begin{{table*}}
#\\small%
#\\classiccaptionstyle%
#\\centering%
#\\input{{\\dataDir/param_{0}}}%
#\\caption{{PARAM {0}}}%
#\\label{{TODO}}
#\\end{{table*}}
#'''.format(name))